In [3]:
# distribute for p2wpkh PSBT
import requests
from ecc import PrivateKey
from hd import HDPublicKey
from helper import decode_bech32, decode_base58, hash256, little_endian_to_int
from network import SimpleNode
from psbt import PSBT, NamedPublicKey, NamedHDPublicKey
from script import P2PKHScriptPubKey, P2WPKHScriptPubKey, RedeemScript, WitnessScript
from tx import Tx, TxIn, TxOut
passphrase = b'jimmy@programmingblockchain.com secret passphrase'
secret = little_endian_to_int(hash256(passphrase))
private_key = PrivateKey(secret)
my_address = private_key.point.bech32_address(testnet=True)
my_hd = HDPublicKey.parse('tpubDCbMpPR2Kvc9Q1hB4VinDyzudyt6MvsTNH2J7LFa421g4J76MYJMMLPgqaa6xU2mowzQg5wVggZrfafPzs385KZSS9xze3hw9UMWgBSHV4g')
target_amount = 1000000
response = requests.get('https://blockstream.info/testnet/api/address/{}/utxo'.format(my_address))
utxos = response.json()
if (len(utxos) == 0):
    raise RuntimeError('no outputs!')
student_xpubs = '''tpubDD9FvAgZq26Q48SMWpzo432dw5eRun268zABpSYC9wbptYM9zR2DYbBygRCd1TQwPNVC88qNgHf3GcoCC15Fx61C7P5sTR6qaUQycpTetBJ'''
fee = 1000
tx_ins = []
total = 0
for utxo in utxos:
    prev_tx = bytes.fromhex(utxo['txid'])
    prev_index = utxo['vout']
    total += utxo['value']
    # create a new tx input
    tx_ins.append(TxIn(prev_tx, prev_index))
# initialize outputs
tx_outs = []
output_amount = 0
path = "m/0/0"
path_2 = "m/0/1"
my_sec_1 = my_hd.traverse(path).sec()
my_sec_2 = my_hd.traverse(path_2).sec()
for student_xpub in student_xpubs.split():
    student_hd = HDPublicKey.parse(student_xpub)
    student_sec = student_hd.traverse(path).sec()
    student_h160 = student_hd.traverse(path).hash160()
    p2wpkh_script_pubkey = P2WPKHScriptPubKey(student_h160)
    # convert target amount to satoshis (multiply by 100 million)
    output_amount += target_amount
    # create a new tx output for target
    tx_outs.append(TxOut(
        amount=target_amount,
        script_pubkey=p2wpkh_script_pubkey,
    ))
    redeem_script = RedeemScript([0, student_h160])
    p2sh_script_pubkey = redeem_script.script_pubkey()
    # convert target amount to satoshis (multiply by 100 million)
    output_amount += target_amount
    # create a new tx output for target
    tx_outs.append(TxOut(
        amount=target_amount,
        script_pubkey=p2sh_script_pubkey,
    ))
    witness_script = WitnessScript([0x52, my_sec_1, student_sec, 0x52, 0xae])
    p2wsh_script_pubkey = witness_script.script_pubkey()
    # convert target amount to satoshis (multiply by 100 million)
    output_amount += target_amount
    # create a new tx output for target
    tx_outs.append(TxOut(
        amount=target_amount,
        script_pubkey=p2wsh_script_pubkey,
    ))
    witness_script = WitnessScript([0x53, my_sec_1, my_sec_2, student_sec, 0x53, 0xae])
    redeem_script = RedeemScript([0, witness_script.sha256()])
    p2sh_script_pubkey = redeem_script.script_pubkey()
    output_amount += target_amount
    # create a separate tx output for target
    tx_outs.append(TxOut(
        amount=target_amount,
        script_pubkey=p2sh_script_pubkey,
    ))
change_satoshis = total - fee - output_amount
my_script_pubkey = P2WPKHScriptPubKey(private_key.point.hash160())
tx_outs.append(TxOut(amount=change_satoshis, script_pubkey=my_script_pubkey))
tx_obj = Tx(version=1, tx_ins=tx_ins, tx_outs=tx_outs, locktime=0, testnet=True, segwit=True)
for i in range(len(tx_obj.tx_ins)):
    tx_obj.sign_input(i, private_key)
if tx_obj.fee() > 0.05*100000000 or tx_obj.fee() <= 0:
    raise RuntimeError('Check that the change amount is reasonable. Fee is {}'.format(tx_obj.fee()))
print(tx_obj.serialize().hex())
node = SimpleNode('testnet.programmingbitcoin.com', testnet=True)
node.handshake()
node.send(tx_obj)
if node.is_tx_accepted(tx_obj):
    print('success! {}'.format(tx_obj.id()))
prev_tx = tx_obj.hash()

3045022100a9968c79bdf4d18534995614864109646f0525dba7f557bc840dcadc4ac9fd07022075167f8cbbade9722b08ab4ccf297acf8ce4d47d1f2c61eadc21f453d274fd7e01 031dbe3aff7b9ad64e2612b8b15e9f5e4a3130663a526df91abfb7b1bd16de5d6e 
010000000001016f0a34017a8777b9b68178ef19553cb35ee8f8087fc1513d4e78f1d7908428ff0300000000ffffffff0540420f0000000000160014f0cd79383f13584bdeca184cecd16135b8a79fc240420f000000000017a914990dd86ae46c3d568535e5e482ac35151836d3cd8740420f0000000000220020c1b4fff485af1ac26714340af2e13d2e89ad70389332a0756d91a123c7fe7f5d40420f000000000017a91423358e259fbcf478331138ceb9619d9a8c83507387d8230301000000001600146e13971913b9aa89659a9f53d327baa8826f2d7502483045022100ffacfbf5a4031fa50a812e79fdc0bfa20707582fe71a4155d13fc1914d838ffd02207e2037242f90130bd8719defc4dae9cd4f1fa16f41fa22d67098f953b12843260121031dbe3aff7b9ad64e2612b8b15e9f5e4a3130663a526df91abfb7b1bd16de5d6e00000000
success! 06f95937a4c72dfa9a2c112d529d39fbf83b4ccb14f1e03923f6abc21d19895c


In [20]:
# create txs for students to create p2pkh and p2sh PSBTs
from ecc import PrivateKey
from hd import HDPrivateKey, HDPublicKey
from helper import encode_varstr, serialize_binary_path, little_endian_to_int, hash256
from io import BytesIO
from psbt import NamedHDPublicKey, PSBT
from script import P2WPKHScriptPubKey, WitnessScript, RedeemScript
from tx import Tx, TxIn, TxOut
prev_tx = bytes.fromhex('06f95937a4c72dfa9a2c112d529d39fbf83b4ccb14f1e03923f6abc21d19895c')
passphrase = b'jimmy@programmingblockchain.com secret passphrase'
secret = little_endian_to_int(hash256(passphrase))
private_key = PrivateKey(secret)
student_xpubs = '''tpubDD9FvAgZq26Q48SMWpzo432dw5eRun268zABpSYC9wbptYM9zR2DYbBygRCd1TQwPNVC88qNgHf3GcoCC15Fx61C7P5sTR6qaUQycpTetBJ'''
mnemonic = 'abandon abandon abandon abandon abandon abandon abandon abandon abandon abandon abandon about'
passphrase = b'jimmy@programmingblockchain.com Jimmy Song'
my_hd_priv = HDPrivateKey.from_mnemonic(mnemonic, passphrase, testnet=True)
my_hd = HDPublicKey.parse('tpubDCbMpPR2Kvc9Q1hB4VinDyzudyt6MvsTNH2J7LFa421g4J76MYJMMLPgqaa6xU2mowzQg5wVggZrfafPzs385KZSS9xze3hw9UMWgBSHV4g')
my_script_pubkey = P2WPKHScriptPubKey(private_key.point.hash160())
path = "m/0/0"
path_2 = "m/0/1"
my_sec_1 = my_hd.traverse(path).sec()
my_sec_2 = my_hd.traverse(path_2).sec()
print(my_hd_priv.traverse("m/44'/1'/0'/0/0").wif())
print(my_hd_priv.traverse("m/44'/1'/0'/0/1").wif())
path = "m/44'/1'/0'"
input_path = "m/0/0"
change_path = "m/1/0"
my_input_sec = my_hd.traverse(input_path).sec()
my_change_sec = my_hd.traverse(change_path).sec()
my_raw_hd = my_hd.raw_serialize()
key = b'\x01' + my_raw_hd
raw_path = bytes.fromhex('fbfef36f') + serialize_binary_path(path)
my_named_hd = NamedHDPublicKey.parse(key, BytesIO(encode_varstr(raw_path)))
my_named_hd_pub = NamedHDPublicKey.from_hd_priv(my_hd_priv, path)
pubkey_lookup = my_named_hd_pub.bip44_lookup()
for i, student_xpub in enumerate(student_xpubs.split()):
    student_hd = HDPublicKey.parse(student_xpub)
    student_input_sec = student_hd.traverse(input_path).sec()
    student_change_sec = student_hd.traverse(change_path).sec()
    student_h160 = student_hd.traverse(change_path).hash160()
    print(student_h160.hex())
    # p2wpkh PSBT
    tx_in = TxIn(prev_tx, 4*i)
    student_script_pubkey = P2WPKHScriptPubKey(student_h160)
    tx_out = TxOut(999000, student_script_pubkey)
    tx_obj = Tx(1, [tx_in], [tx_out], 0, testnet=True)
    result = tx_obj.serialize().hex()
    # p2sh-p2wpkh PSBT
    tx_in = TxIn(prev_tx, 4*i+1)
    tx_out = TxOut(999000, my_script_pubkey)
    tx_obj = Tx(1, [tx_in], [tx_out], 0, testnet=True)
    result += '\t{}'.format(tx_obj.serialize().hex())
    # p2wsh PSBT
    witness_input = WitnessScript([0x52, my_input_sec, student_input_sec, 0x52, 0xae])
    witness_change = WitnessScript([0x52, my_change_sec, student_change_sec, 0x52, 0xae])
    tx_in = TxIn(prev_tx, 4*i+2)
    change_script_pubkey = witness_change.script_pubkey()
    tx_out = TxOut(999000, change_script_pubkey)
    psbt_obj = PSBT.create(Tx(1, [tx_in], [tx_out], 0, testnet=True))
    psbt_obj.hd_pubs[my_raw_hd] = my_named_hd
    result += '\t{}\t{}\t{}'.format(witness_input.serialize().hex(), witness_change.serialize().hex(), psbt_obj.serialize().hex())
    # p2sh-p2wsh PSBT
    tx_in = TxIn(prev_tx, 4*i+3)
    psbt_obj = PSBT.create(Tx(1, [tx_in], [tx_out], 0, testnet=True))
    tx_lookup = psbt_obj.tx_obj.get_input_tx_lookup()
    witness_input_2 = WitnessScript([0x53, my_sec_1, my_sec_2, student_input_sec, 0x53, 0xae])
    witness_script_lookup = {witness_input_2.sha256(): witness_input_2}
    redeem_input = RedeemScript([0, witness_input_2.sha256()])
    redeem_script_lookup = {redeem_input.hash160(): redeem_input}
    psbt_obj.update(tx_lookup, pubkey_lookup, redeem_script_lookup, witness_script_lookup)
    psbt_obj.sign(my_hd_priv)
    keys = psbt_obj.psbt_ins[0].sigs.keys()
    for key in keys:
        del psbt_obj.psbt_ins[0].sigs[key]
        result += '\t{}'.format(psbt_obj.serialize().hex())
        psbt_obj.sign(my_hd_priv)
    print(result)

cP88EsR4DgJNeswxecL4sE4Eornf3q1ZoRxoCnk8y9eEkQyxu3D7
cP9BYGBfMbhsN5Lvyza3otuC14oKjqHbgbRXhm7QCF47EgYWQb6S
27459b7e4317d1c9e1d0f8320d557c6bb08731ef
01000000015c89191dc2abf62339e0f114cb4c3bf8fb399d522d112c9afa2dc7a43759f9060000000000ffffffff01583e0f000000000016001427459b7e4317d1c9e1d0f8320d557c6bb08731ef00000000	01000000015c89191dc2abf62339e0f114cb4c3bf8fb399d522d112c9afa2dc7a43759f9060100000000ffffffff01583e0f00000000001600146e13971913b9aa89659a9f53d327baa8826f2d7500000000	47522102c1b6ac6e6a625fee295dc2d580f80aae08b7e76eca54ae88a854e956095af77c210247aed77c3def4b8ce74a8db08d7f5fd315f8d96b6cd801729a910c3045d750f252ae	47522102db8b701c3210e1bf6f2a8a9a657acad18be1e8bff3f7435d48f973de8408f29021026421c7673552fdad57193e102df96134be00649195b213fec9d07c6d918f418d52ae	70736274ff01005e01000000015c89191dc2abf62339e0f114cb4c3bf8fb399d522d112c9afa2dc7a43759f9060200000000ffffffff01583e0f0000000000220020878ce58b26789632a24ec6b62542e5d4e844dee56a7ddce7db41618049c3928c000000004f01043587cf034d513c158000000

In [7]:
# sign first tx p2sh 1 PSBTs
from hd import HDPrivateKey
student_psbts = '''70736274ff01005e01000000015c89191dc2abf62339e0f114cb4c3bf8fb399d522d112c9afa2dc7a43759f9060200000000ffffffff01583e0f0000000000220020878ce58b26789632a24ec6b62542e5d4e844dee56a7ddce7db41618049c3928c000000004f01043587cf034d513c1580000000fb406c9fec09b6957a3449d2102318717b0c0d230b657d0ebc6698abd52145eb02eaf3397fea02c5dac747888a9e535eaf3c7e7cb9d5f2da77ddbdd943592a14af10fbfef36f2c00008001000080000000800001012b40420f0000000000220020c1b4fff485af1ac26714340af2e13d2e89ad70389332a0756d91a123c7fe7f5d22020247aed77c3def4b8ce74a8db08d7f5fd315f8d96b6cd801729a910c3045d750f247304402204fd654c27002d4c9e53bb001229e3d7587e5be245a81b6f7ead3bf136643af40022060ebf1193a6b3e82615a564f0043e5ae88e661bfdb7fd254c9a30bae8160583901010547522102c1b6ac6e6a625fee295dc2d580f80aae08b7e76eca54ae88a854e956095af77c210247aed77c3def4b8ce74a8db08d7f5fd315f8d96b6cd801729a910c3045d750f252ae22060247aed77c3def4b8ce74a8db08d7f5fd315f8d96b6cd801729a910c3045d750f218797dcdac2c00008001000080000000800000000000000000220602c1b6ac6e6a625fee295dc2d580f80aae08b7e76eca54ae88a854e956095af77c18fbfef36f2c0000800100008000000080000000000000000000010147522102db8b701c3210e1bf6f2a8a9a657acad18be1e8bff3f7435d48f973de8408f29021026421c7673552fdad57193e102df96134be00649195b213fec9d07c6d918f418d52ae2202026421c7673552fdad57193e102df96134be00649195b213fec9d07c6d918f418d18797dcdac2c00008001000080000000800100000000000000220202db8b701c3210e1bf6f2a8a9a657acad18be1e8bff3f7435d48f973de8408f29018fbfef36f2c0000800100008000000080010000000000000000'''
for student_psbt in student_psbts.split():
    psbt_obj = PSBT.parse(BytesIO(bytes.fromhex(student_psbt)))
    print(psbt_obj.sign(my_hd_priv))
    print(psbt_obj.serialize().hex())

True
70736274ff01005e01000000015c89191dc2abf62339e0f114cb4c3bf8fb399d522d112c9afa2dc7a43759f9060200000000ffffffff01583e0f0000000000220020878ce58b26789632a24ec6b62542e5d4e844dee56a7ddce7db41618049c3928c000000004f01043587cf034d513c1580000000fb406c9fec09b6957a3449d2102318717b0c0d230b657d0ebc6698abd52145eb02eaf3397fea02c5dac747888a9e535eaf3c7e7cb9d5f2da77ddbdd943592a14af10fbfef36f2c00008001000080000000800001012b40420f0000000000220020c1b4fff485af1ac26714340af2e13d2e89ad70389332a0756d91a123c7fe7f5d220202c1b6ac6e6a625fee295dc2d580f80aae08b7e76eca54ae88a854e956095af77c47304402203f26a975aae04a7ae12c964cdcea318c850351a3072aebbab7902e89957008ea022019f895271f70d1515f9da776d6ac17c21bcbca769d87c1beb4ebbf4c7a56fbc20122020247aed77c3def4b8ce74a8db08d7f5fd315f8d96b6cd801729a910c3045d750f247304402204fd654c27002d4c9e53bb001229e3d7587e5be245a81b6f7ead3bf136643af40022060ebf1193a6b3e82615a564f0043e5ae88e661bfdb7fd254c9a30bae8160583901010547522102c1b6ac6e6a625fee295dc2d580f80aae08b7e76eca54ae88a854e956095af77